Getting the number of pages in the PDF document. I altered the original PDFMiner library in tools/pdf2txt.py so that with the extra -Q argument, the script returns the no of pages.

In [48]:
import subprocess
import re
import tabula
import nltk
from os import remove as DeleteFile
from os import listdir
from os.path import isfile, join
import pandas as pd
from PyPDF2 import PdfFileReader
#getting file names from docs
mypath = './docs'
files_docs = [f for f in listdir(mypath) if isfile(join(mypath, f))]
file = files_docs[9]
print(files_docs)
#checking if name meets standards for later printing use
pattern = re.compile("^\d{2,3}\. [\S\s]+? \([A-Za-z ]+\) \d{4} ?Q?[1-4]?")
match = pattern.match(file)
if (match):
    title = match.string
    print(title)
path = "./docs/" + file
OFFSET = int(file[0:2])
OFFSET_STRING = str(OFFSET)+','+str(OFFSET+1)+','+str(OFFSET+2)

cmd = ['python2', './pdf2txt/tools/pdf2txt.py', '-p', OFFSET_STRING, '-t', 'text', '-Q', path]
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
temp, error = p.communicate()
if error:
    print(error.decode("utf-8"))
pages = int(temp.decode("utf-8"))
print('Number of pages: ', pages)

['29. Northern Trust Asset Mgmt. (US) 2010.pdf', '12. BlackRock (US) 2009.pdf', '10. Rowe Price (US) 2012.pdf', '91. Allianz Group (Germany) 2010.pdf', "99. but in fact it's 108 J.P. Morgan Chase (US) 2013.pdf", '08. Templeton Investments (US) 2009.pdf', '37. Bank of New York Mellon (US) 2015.pdf', '11. Schroders Investment Managment (UK) 2012.pdf', '51. Goldman Sachs (US) 2010.pdf', '08. Invesco (US) 2010.pdf', '53. Deutsche Bank (Germany) 2012.pdf']
08. Invesco (US) 2010.pdf
Number of pages:  3


I load the pages into the document object.

In [49]:
document = {}
for page in range(pages):
    print("p",page, " is loading")
    document[OFFSET+page] = ''
    #cmd = ["python2 ./pdf2txt/tools/pdf2txt.py -o ./temp/output"+str(page)+".txt -p "+ str(OFFSET+page) +" -t text '" + path+"'"]
    #subprocess.call(cmd, shell = True)
    cmd = ["python2", "./pdf2txt/tools/pdf2txt.py", "-o", "./temp/output"+str(OFFSET+page)+".txt", "-p", str(OFFSET+page),"-t", "text", path]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    temp, error = p.communicate()
    if error:
        print(error.decode("utf-8"))
        
    with open("./temp/output"+str(page)+".txt", "r") as page_output:
        for line in page_output:
            document[OFFSET+page] += line.strip().lower() + ' '
    page_output.close()
#DeleteFile('./output.txt')    
print("Document loaded")

p 0  is loading
p 1  is loading
p 2  is loading
Document loaded


Once loaded, the document has to go through a basic page scoring step. If the page doesn't contain Asset/Investment allocation or the two words equity and fixed income, then the page is thrown away.

In [50]:
page_nomination = []
case1 = "(?:asset|investments?)\sallocation"
case2 = "equit(?:y|ies)[\s\S]*?fixed\sincome"
case3 = "fixed\sincome[\s\S]*?equit(?:y|ies)"
cases = "(?:"+case1+")|(?:"+case2+")|(?:"+case3+")"
p = re.compile(cases)
for page in document:
    matches = p.findall(document[page])
    if matches:
        page_nomination.append(page)
    print(matches)
#taking subset of original document object
document = {k: document[k] for k in page_nomination}


[('', '', 'equities, 28% in fixed income', 'ies', '', '')]
[('', '', 'equity and multi- asset class offerings accounting for 47% and 5% of total emea client aum, respectively, and our relatively newer fixed income', 'y', '', ''), ('', '', 'equities, 41% in fixed income', 'ies', '', ''), ('', '', 'equities, 38% in fixed income', 'ies', '', ''), ('', '', 'equities, fixed income', 'ies', '', '')]
[('', '', 'equities, 24%, or $93.2 billion, in fixed income', 'ies', '', ''), ('', '', 'equity and fixed income', 'y', '', '')]


To analyze the pages, first I'll try to convert it to a table and then I'll validate the first table column entries (equity, fixed income, and optionally multi asset and alternatives) I'll also try to look for columns indicating years.

In [76]:
found_table = False
input1 = PdfFileReader(open(path, "rb"))
try:
    page = input1.getPage(0)
except:
    input1.decrypt('')
    page = input1.getPage(0)

top = page.mediaBox.upperRight[1]
right = page.mediaBox.upperRight[0]
bottom = page.mediaBox.lowerLeft[1]
left = page.mediaBox.lowerLeft[0]
areas = [[top, left, bottom, right],
         [top, left, (bottom+top)/2, right],[(bottom+top)/2, left, bottom, right],
         [top, left, (bottom+top)/2, (right+left)/2],[top, (right+left)/2, (bottom+top)/2, right],
         [(bottom+top)/2, (right+left)/2, bottom, right],[(bottom+top)/2, left, bottom, (right+left)/2]
        ]

k = re.compile('allocation|fixed income|equity|equities|hybrid|[Aa]lternative|multi[- ]asset|cash management|money market', re.IGNORECASE)
j = re.compile('(?:USD|US dollars?|dollars?|[$¢£¤¥֏؋৲৳৻૱௹฿៛\u20a0-\u20bd\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6])?\d+(?:[ ,\.-]?\d+){0,3}(?:million|thousand|hundred|billion|M|B|T|K|m|bn|tn)?(?:USD|US dollars?|dollars?|[$¢£¤¥֏؋৲৳৻૱௹฿៛\u20a0-\u20bd\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6])?', re.IGNORECASE)
df = pd.DataFrame([], columns=['equities', 'equities %', 'fixed income', 'fixed income %', 'alternative', 'alternative %', 'multi-asset','multi-asset %', 'cash management', 'cash management %'], index=[title])

for page in [OFFSET]:#document:#
    for  area in areas:
        try:
            table = tabula.read_pdf(path, pages=page, area = area)
            p = table.T.reset_index().T
            p = p.apply(lambda x: x.astype(str).str.lower())
            for row_ind, row in p.iterrows():
                search = True
                for col_ind, col in row.iteritems():
                    if search: 
                        if (len(k.findall(col))>1):
                            break
                        if bool(k.search(col)):
                            asset = k.search(col).group(0)
                            search = False
                            
                            if bool(j.search(col)):
                                df[asset] = j.search(col).group(0)
                                break
                    else:
                        if bool(j.search(col)):
                            df[asset] = j.search(col).group(0)
                            break
                        
            if (df.T.count()[0] > 2):
                print('Result found, saving p', page)
                found_table = True
                print(df.T)
                break
            else:
                print('Invalid values')            
        except Exception as inst:
            print("Unsuccessful table search: page", page, inst)


    

Unsuccessful table search: page 8 Error tokenizing data. C error: Expected 5 fields in line 8, saw 7

equity equity $294.1 <_sre.SRE_Match object; span=(7, 13), match='$294.1'>
money market money market $68.3 <_sre.SRE_Match object; span=(13, 18), match='$68.3'>
fixed income fixed income $131.9 <_sre.SRE_Match object; span=(13, 19), match='$131.9'>
alternative alternative None
Result found, saving p 8
                  08. Invesco (US) 2010.pdf
equities                                NaN
equities %                              NaN
fixed income                         $131.9
fixed income %                          NaN
alternative                             NaN
alternative %                           NaN
multi-asset                             NaN
multi-asset %                           NaN
cash management                         NaN
cash management %                       NaN
equity                               $294.1
money market                          $68.3                    ($ b

If no tables are found, we continue trying to text mine the expected results. This Python2 script looks for conversational sentences.

In [11]:
for page in document:
    text = ''
    with open("./temp/output"+str(page)+".txt", "r") as inputfile:
        for line in inputfile:
            text += line

    h = re.compile('[A-Z][^\.][\s\S]*?[^A-Z][.?!](?![A-Z][^a-z]|\d)')
    matches = h.findall(text)
    print(matches)
    #looking for sentences that might qualify
    to_drop = []
    j = re.compile('allocation|fixed income|equity|equities|hybrid|multi-assets?|alternatives|mix|management|assets|investment|managed|invested?', re.IGNORECASE)
    k = re.compile('allocation|fixed income|equity|equities|hybrid|alternatives|multi-assets?', re.IGNORECASE)
    for i in range(len(matches)-1, -1, -1):
        matches[i] = matches[i].replace('\n', ' ').replace('\r', '')
        length = 0 if bool(j.search(matches[i])) else len(j.findall(matches[i]))
        if(length < 2 and (not bool(k.search(matches[i])) or not bool(re.search(r'\d', matches[i])))):
            to_drop.append(i)

    for i in to_drop:
        matches.pop(i)

    print(matches)


['Portfolios may be invested in local, regional or global capital markets.', 'Products may be structured to address location-specific issues, such as regulations, taxation, operational infra-structure, and market liquidity, and client-specific issues, such as investment policy, liability structure and ratings.', 'At year-end 2009, assets managed on behalf of clients domiciled in the United States and Canada (including offshore investors), totaled $2.042 trillion or 61% of total AUM, an increase of $1.070 trillion versus reported AUM at December 31, 2008.', 'On a pro forma combined basis, AUM increased 19% or $322.3 billion, including $62.1 billion of net new business and $264.4 billion from invest-ment performance and market appreciation.', 'Offerings include closed-end funds and iShares traded on domestic stock exchanges, a full range of open-end mutual funds, collective investment funds, common trusts, private funds and separate accounts.', 'Clients invest across the full product ran

In [19]:
#regexs to distinguish dollar values, asset classes and percent values
digits = "\d+(?:[ ,\.-]?\d+){0,3}"
currency = "(?:USD|US dollars?|dollars?|[$¢£¤¥֏؋৲৳৻૱௹฿៛\u20a0-\u20bd\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6])"
words = "(?:(?:million|thousand|hundred|billion|M|B|T|K|m|bn|tn)(?![a-z]))"

classes = '(?:equities|equity|fixed income|alternatives?|multi[ -]assets?|hybrid|cash management)'
money = currency+'?\s?'+digits+'\s?'+words+'?\s?'+currency+'?(?![%\d])'
percent = digits+'%'

#tokenize into the useful tokens and tag them whether they're nubmers or classes blabla
pattern = r''+percent+'|'+classes+'|'+money
tokenizer = nltk.tokenize.RegexpTokenizer(pattern)
tokens = tokenizer.tokenize(matches[0])
regexp_tagger = nltk.tag.RegexpTagger(
    [(r''+classes, 'CL'),
     (r''+money, 'NU'),
     (r''+percent, 'PC'),     
    ])
tags = regexp_tagger.tag(tokens)
print(tags)

#we only use the sentence if the number of asset classes match up with the number values. Otherwise it's just a mess
pc = 0
nu = 0
cl = 0
for i in tags:
    if i[1] == 'PC':
        pc += 1
    elif i[1] == 'NU':
        nu += 1
    elif i[1] == 'CL':
        cl += 1

df = pd.DataFrame([], columns=['equities', 'equities %', 'fixed income', 'fixed income %', 'alternative', 'alternative %', 'multi-asset','multi-asset %', 'cash management', 'cash management %'], index=[title])

per = None
num = None
cla = None
if (pc == cl == nu):
    for i in tags:        
        if i[1] == 'PC':
            if per == None:
                per = i[0]
            else:
                print("Values didn't match up. Sentence not finished.")
                break
        if i[1] == 'NU':
            if num == None:
                num = i[0]
            else:
                print("Values didn't match up. Sentence not finished.")
                break
        if i[1] == 'CL':
            if cla == None:
                cla = i[0]
            else:
                print("Values didn't match up. Sentence not finished.")
                break
        if per and num and cla:
            df[cla] = num
            df[cla+' %'] = per
            per = num = cla = None
            
elif (cl == pc):
    for i in tags:        
        if i[1] == 'PC':
            if per == None:
                per = i[0]
            else:
                print("Values didn't match up. Sentence not finished.")
                break
        if i[1] == 'CL':
            if cla == None:
                cla = i[0]
            else:
                print("Values didn't match up. Sentence not finished.")
                break
        if per and cla:
            df[cla+' %'] = per
            cla = per = None
elif (cl == nu):
    for i in tags:        
        if i[1] == 'NU':
            if num == None:
                num = i[0]
            else:
                print("Values didn't match up. Sentence not finished.")
                break
        if i[1] == 'CL':
            if cla == None:
                cla = i[0]
            else:
                print("Values didn't match up. Sentence not finished.")
                break
        if num and cla:
            df[cla] = num
            num = cla = None
print(df.T)         

[('40%', 'PC'), ('$153.9 billion', 'NU'), ('equities', 'CL'), ('24%', 'PC'), ('$93.2 billion', 'NU'), ('fixed income', 'CL'), ('17%', 'PC'), ('multi-asset', 'CL'), ('$65.2 billion', 'NU'), ('cash management', 'CL'), ('$67.6 billion', 'NU'), ('2%', 'PC'), ('$8.9 billion', 'NU'), ('alternative', 'CL')]
                  12. BlackRock (US) 2009.pdf
equities                       $153.9 billion
equities %                                NaN
fixed income                    $93.2 billion
fixed income %                            NaN
alternative                      $8.9 billion
alternative %                             NaN
multi-asset                     $65.2 billion
multi-asset %                             NaN
cash management                 $67.6 billion
cash management %                         NaN


If we still haven't found results, we simply looks for words like fixed income and try to see if there is a number next to then that could be the value we are looking for.